In [2]:
import pandas as pd
import numpy as np
import pyodbc as pyo
import os
import geopandas as gpd
import shapely
import fiona
from pivottablejs import pivot_ui
from IPython.display import display
import openpyxl
# import plotly.express as px
from shapely.geometry import Point, LineString, Polygon
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
%%time
cnxn = pyo.connect('DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={};'.format(os.path.join(os.getcwd(),'2017-20 pooled SEQ QTS.accdb')))
cursor = cnxn.cursor();tabs = cursor.tables();tablelist = []
for i in tabs:
    if i.table_name.startswith("MSys"):
        pass
    else:
#         print(i.table_name)
        tablelist.append(i.table_name)
del i, tablelist, tabs

Wall time: 9.73 s


In [4]:
%%time
"""Import HTS Data."""

HOUSEHOLDS = pd.read_sql('SELECT * from {}'.format('1_QTS_HOUSEHOLDS'), con = cnxn).fillna(0)
PERSONS = pd.read_sql('SELECT * from {}'.format('2_QTS_PERSONS'), con = cnxn).fillna(0)
#VEHICLES = pd.read_sql('SELECT * from {}'.format('3_QTS_VEHICLES'), con = cnxn).fillna(0)
STOPS = pd.read_sql('SELECT * from {}'.format('4_QTS_STOPS'), con = cnxn).fillna(0)
TRIPS = pd.read_sql('SELECT * from {}'.format('5_QTS_TRIPS'), con = cnxn).fillna(0)
ANZSCO = pd.read_sql('SELECT * from {}'.format('R_ANZSCO'), con = cnxn).fillna(0)
ASGS = pd.read_sql('SELECT * from {}'.format('R_ASGS_2016'), con = cnxn).fillna(0)
#DIST_bin = pd.read_sql('SELECT * from {}'.format('R_Distance_bins'), con = cnxn).fillna(0)
#MAINACT = pd.read_sql('SELECT * from {}'.format('R_GROUP_MAINACT'), con = cnxn).fillna(0)
#GRP_MODE = pd.read_sql('SELECT * from {}'.format('R_GROUP_MODE'), con = cnxn).fillna(0)
#LGA = pd.read_sql('SELECT * from {}'.format('R_LGA'), con = cnxn).fillna(0)
#MAINMODE = pd.read_sql('SELECT * from {}'.format('R_MAINMODE'), con = cnxn).fillna(0)
#PURPOSE = pd.read_sql('SELECT * from {}'.format('R_OVERALL_PURPOSE'), con = cnxn).fillna(0)
#REGION = pd.read_sql('SELECT * from {}'.format('R_REGION'), con = cnxn).fillna(0)

TIME = pd.read_excel('R_TIME.xlsx', engine = 'openpyxl').fillna(0)

Wall time: 23.4 s


In [5]:
# """Import Toowoomba HTS Data."""
# twb_cnxn = pyo.connect('DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={};'.format(os.path.join(os.getcwd(),'2019-20 TWB QTS V1.1.accdb')))

# twb_HOUSEHOLDS = pd.read_sql('SELECT * from {}'.format('1_QTS_HOUSEHOLDS'), con = twb_cnxn).fillna(0)
# twb_PERSONS = pd.read_sql('SELECT * from {}'.format('2_QTS_PERSONS'), con = twb_cnxn).fillna(0)
# twb_STOPS = pd.read_sql('SELECT * from {}'.format('4_QTS_STOPS'), con = twb_cnxn).fillna(0)
# twb_TRIPS = pd.read_sql('SELECT * from {}'.format('5_QTS_TRIPS'), con = twb_cnxn).fillna(0)

# PERSONS = pd.concat([PERSONS, twb_PERSONS])
# HOUSEHOLDS = pd.concat([HOUSEHOLDS, twb_HOUSEHOLDS])
# STOPS = pd.concat([STOPS, twb_STOPS])
# TRIPS = pd.concat([TRIPS, twb_TRIPS])

# del twb_HOUSEHOLDS, twb_PERSONS,twb_STOPS, twb_TRIPS 

In [6]:
"""Adjust PERSID."""
PERSONS['PERSID'] = PERSONS['PERSID'].str.replace('/','')
TRIPS['PERSID'] = TRIPS['PERSID'].str.replace('/','')
STOPS['PERSID'] = STOPS['PERSID'].str.replace('/','')

In [7]:
PERSONS.drop(columns = ['STARTPLACE', 'ANYSTOPS', 'REASON', 'LASTLEFT','RELATIONSHIP', 'CARLICENCE', 'CARLICTYPE',
                        'MCLICENCE', 'MCLICTYPE', 'OTHERLICENCE','YEAROFBIRTH','ASSISTAGE', 'ASSISTLTHC', 'ASSISTSTHC',
                        'ASSISTDISABILITY','ASSISTENGLISH', 'ASSISTOTHER', 'ASSISTANY','RIDESHARETRIPS', 'RIDESHAREENT', 
                        'RIDESHAREHC', 'RIDESHAREED', 'RIDESHARESHOP', 'RIDESHAREWORK', 'RIDESHAREOTHER', 'TAXITRIPSS',
                        'TAXIENT', 'TAXIHC', 'TAXIED', 'TAXISHOP', 'TAXIWORK', 'TAXIOTHER'], 
             inplace = True)

HOUSEHOLDS.drop(columns= ['BIKES', 'HHVEH', 'DWELLTYPE', 'SURVEYWEEK',
       'STRATA_LGA', 'TRAVDATE', 'TRAVMONTH', 'TRAVYEAR', 'TRAVDOW'], inplace = True)

In [8]:
"""Load New Weightings"""
HH = pd.read_excel('Reweight.xlsx', engine='openpyxl', sheet_name='HH_export')
Pers = pd.read_excel('Reweight.xlsx', engine='openpyxl', sheet_name='Pers_export')
wgt_sect = pd.read_excel('Reweight.xlsx', engine='openpyxl', sheet_name='WGT_Sect')

In [9]:
"""Load Spatial Cordons"""
district_path = r'Spatial Files\SEQSTM_Districts_vF.TAB' 
district = gpd.read_file(district_path, crs='EPSG:4283')
district = district.to_crs(epsg=4283)
district = district.astype({"Dist_ID": int})
district['SEQSTM'] = 'Y'

BSTM_path = r"Spatial Files\BSTM_Districts.TAB"
BSTM_district = gpd.read_file(BSTM_path, crs='EPSG:4283')
BSTM_district = BSTM_district.to_crs(epsg=4283)
BSTM_district = BSTM_district.astype({"Dist_ID": int})
BSTM_district['BSTM'] = 'Y'

zones = gpd.read_file(r'Spatial Files\SEQSTM_Zones_wExt_LvL4_vF2_withIslands.TAB')
zones = zones.to_crs(epsg=4283)
zones = zones.astype({'LvL4_ID':int, 'LvL3_ID':int, 'LvL2_ID':int, 'LvL1_ID': int, 'Dist_ID':int})
zones['SEQSTM'] = 'Y'
zones = pd.merge(zones,BSTM_district[['Dist_ID', 'BSTM']], how = 'left', on='Dist_ID').fillna('N')
zones = zones[['geometry','LvL4_ID', 'LvL3_ID', 'LvL2_ID', 'LvL1_ID', 'Dist_ID','SA2_NAME','SEQSTM','BSTM']]
zones = pd.merge(zones, wgt_sect[['Dist_ID','WeightingSector']], on='Dist_ID')
del BSTM_district, district

# Data Table Cleaning

## Households 

In [10]:
Household = HOUSEHOLDS.copy()
Household = Household.astype({'HHID':'int64'})
Household = pd.merge(Household, ASGS, left_on = 'HOME_SA1_2016', right_on = 'SA1_7DIGITCODE_2016')

"""Spatial Join to Model Sectors"""
Household_map = gpd.GeoDataFrame(Household, geometry=gpd.points_from_xy(Household['HOMELONG'], Household['HOMELAT']), crs='EPSG:4283')
Household_map = gpd.sjoin(Household_map, zones[['geometry','LvL4_ID', 'LvL3_ID', 'LvL2_ID', 'LvL1_ID', 'Dist_ID','BSTM','SEQSTM']],
 how = 'left', op = 'within').drop(columns ='index_right')#.astype({'LvL4_ID':int, 'LvL3_ID':int, 'LvL2_ID':int, 'LvL1_ID': int, 'Dist_ID':int})

Household = pd.DataFrame(Household_map).drop(columns = 'geometry')
Household.rename(columns = {'LvL4_ID':'Home_LvL4', 'LvL3_ID':'Home_LvL3', 'LvL2_ID':'Home_LvL2', 'LvL1_ID':'Home_LvL1', 
'Dist_ID':'Home_Dist', 'SA2_NAME_2016':'HH_SA2', 'SA3_NAME_2016': 'HH_SA3', 'SA4_NAME_2016': "HH_SA4", 'LGA_NAME_2016': "HH_LGA"}, inplace = True)

Household.drop(columns=['SA1_7DIGITCODE_2016', 'SA2_5DIGITCODE_2016', 'SA3_CODE_2016', 'SA4_CODE_2016'], inplace=True)

#district_plot = district.plot(figsize=(40,20))
#zone_plot = zones.plot(column = 'BSTM', figsize=(40,20),cmap= 'Pastel2')
#hh_map = Household_map.plot(column = 'LGA_NAME_2016', ax = zone_plot, marker = ",", markersize=2,cmap='OrRd', legend = True)#coolwarm
#hh_map.savefig('HH_map.pdf')
#del Household_map, zone_plot, hh_map

print('Number of Household Records in Original: {}'.format(len(HOUSEHOLDS.axes[0])))
print('Number of Household Records: {}'.format(len(Household.axes[0])))

"""Update Household Weights"""
Household['old_HHWGT_19'] = Household['HHWGT_19']
Household = Household.set_index("HHID")
Household.update(HH.set_index('HHID'))
Household.reset_index(inplace=True)

print("LGA Weighting Population: {}".format(Household.old_HHWGT_19.sum().round(0)))
print("Sector Weighting Population: {}".format(Household.HHWGT_19.sum().round(0)))

del Household_map
Household[['HH_LGA','old_HHWGT_19','HHWGT_19']].groupby(by='HH_LGA').agg(['count', 'sum']).round(0)

C:\Users\Karimi\Anaconda3\envs\imgenv\lib\site-packages\geopandas\tools\sjoin.py:61: UserWarning: CRS of frames being joined does not match!(EPSG:4283 != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


Number of Household Records in Original: 14715
Number of Household Records: 14715
LGA Weighting Population: 1167298.0
Sector Weighting Population: 1166010.0


old_HHWGT_19           HHWGT_19          
                          count       sum    count       sum
HH_LGA                                                      
Brisbane (C)               2963  427770.0     2963  427403.0
Gold Coast (C)             2038  214279.0     2038  218276.0
Ipswich (C)                1747   70732.0     1747   69231.0
Logan (C)                  1702  103359.0     1702  102988.0
Moreton Bay (R)            1896  157796.0     1896  157320.0
Noosa (S)                   956   20877.0      956   23174.0
Redland (C)                1655   54725.0     1655   54598.0
Sunshine Coast (R)         1758  117760.0     1758  113020.0

## Person Table

In [11]:
""" Blue/White Collar Adjustment."""
adjustpath = os.getcwd()
try:
    ANZSCO_filter = pd.read_csv(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))
    print('Using existing Blue/White Collar split table')
    #os.startfile(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))
except IOError:
    print('Rerun after completing Blue/White Collar split table')
    ANZSCO.reindex(ANZSCO.columns.tolist() + ['Collar', 'Collar2'], axis=1).drop(columns=['ANZSCO_2','Description_2',
    'ANZSCO_4','Description_4','ANZSCO_6', 'Description_6']).drop_duplicates().to_csv(os.path.join(adjustpath,'ANZSCO_adjustment.csv'), index= False)
    os.startfile(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))


Using existing Blue/White Collar split table


In [12]:
print(os.path.join(adjustpath, 'ANZSCO_adjustment.csv'))

H:\Transportation Project\Household_travel_survey\notebook_for_documentation (1)\ANZSCO_adjustment.csv


In [13]:
ANZSCO_filter.loc[~pd.isna(ANZSCO_filter.Collar)]

,ANZSCO_1,Description_1,ANZSCO_3,Description_3,Collar,Collar2
0,1,MANAGERS,100,Managers nfd,White,Professional
1,1,MANAGERS,141,Accommodation and Hospitality Managers,White,White
2,1,MANAGERS,149,"Miscellaneous Hospitality, Retail and Service ...",White,White
3,1,MANAGERS,132,Business Administration Managers,White,Professional
4,1,MANAGERS,131,"Advertising, Public Relations and Sales Managers",White,Professional
...,...,...,...,...,...,...
134,8,LABOURERS,891,Freight Handlers and Shelf Fillers,Blue,Blue
135,8,LABOURERS,890,Other Labourers nfd,Blue,Blue
136,9,MISCELLANEOUS,999,Response Unidentifiable,White,White
137,9,MISCELLANEOUS,997,Response Unidentifiable,White,White


In [14]:
Person = PERSONS.copy()

Person = Person.astype({'HHID':'int64', "PERSID": 'int64', 'ANZSCO_1-digit': int, 'ANZSCO_3-digit': 'int64'}).sort_values('HHID')
Person = pd.merge(Person, Household[['HHID', 'HH_SA2', 'HH_SA3', 'HH_SA4', 'HH_LGA',
'Home_LvL4', 'Home_LvL3', 'Home_LvL2', 'Home_LvL1', 'Home_Dist', 'BSTM']], left_on = 'HHID', right_on = 'HHID', how = 'left')

"""Fix Random Errors Here."""
Person.loc[Person.PERSID == 613521001,'ANZSCO_1-digit'] = 2
Person.loc[Person.PERSID == 613521001,'ANZSCO_3-digit'] = 200
#twb
Person.loc[Person.PERSID == 766731002, 'ANZSCO_3-digit'] = 800
Person.loc[Person.PERSID == 766731001, 'ANZSCO_3-digit'] = 141
Person.loc[Person.PERSID == 782701000, 'ANZSCO_3-digit'] = 500

Person.replace(['primary','secondary'],['Primary','Secondary'], inplace=True)
"""Fix Random Errors Here."""

"""Add Collar Tag."""
%time Person = pd.merge(Person, ANZSCO_filter[['ANZSCO_3','Collar', 'Collar2']], left_on = 'ANZSCO_3-digit', right_on = 'ANZSCO_3', how = 'left').drop(columns= 'ANZSCO_3').sort_values('HHID').drop_duplicates().dropna(axis = 0, thresh = 5)

Person = Person.reset_index().drop(columns = 'index')
Person = Person.astype({'HHID':'int64', "PERSID": 'int64', 'ANZSCO_1-digit': int, 'ANZSCO_3-digit': 'int64'}).sort_values('HHID')

"""Update New Person Weights."""
Person['old_PERSWGT19'] = Person['PERSWGT19']
Person = Person.set_index("PERSID")
Person.update(Pers.set_index('PERSID'))
Person.reset_index(inplace=True)

print('Number of Person Records in Original: {}'.format(len(PERSONS.axes[0])))
print('Number of Person Records: {}'.format(len(Person.axes[0])))

Wall time: 313 ms
Number of Person Records in Original: 36264
Number of Person Records: 36264


### Person Variables

In [15]:
"""Calculate Kindergarten/Childcare kids."""
trip_rev = TRIPS.copy()
kindy_pers = Person.copy()

"""Age Bands."""
under5 = kindy_pers.AGE < 5

"""Processing."""
duration = trip_rev.DURATION > 90
DEST_CC_kindy = trip_rev.DESTPLACE == 'Childcare or kindergarten'

CC_person = trip_rev[['PERSID',"HHID", "DESTPLACE", "DURATION"]].loc[duration & DEST_CC_kindy,['PERSID']].drop_duplicates().astype({'PERSID': 'int64'})
CC_person['CC/Kindy'] = 1

kindy_pers = pd.merge(kindy_pers, CC_person, on='PERSID', how='left')
CC_person = kindy_pers["CC/Kindy"] == 1
not_CC_kid = kindy_pers.STUDYING != 'CC/Kindy'

kindy_pers.loc[CC_person & under5 & not_CC_kid,'STUDYING'] = 'CC/Kindy'
kindy_pers.loc[CC_person & under5 & not_CC_kid,'Student_Prim'] = 0
kindy_pers.loc[CC_person & under5 & not_CC_kid,'Student_CC'] = 1
print("Number of Added Kindy Kids:{}".format(len(kindy_pers.loc[CC_person & under5 & not_CC_kid])))
kindy_pers.drop(columns=['CC/Kindy'], inplace = True)

#Assign Kindy
kindy = kindy_pers.STUDYING == 'CC/Kindy'
kindy_pers.loc[kindy, 'Student_CC'] = 1

Person = kindy_pers
del trip_rev, kindy_pers, kindy, not_CC_kid, under5, duration, DEST_CC_kindy, CC_person


Number of Added Kindy Kids:368


In [16]:
test = Person.loc[Person.Student_CC==1]
test.HH_SA4.value_counts()

Gold Coast             64
Sunshine Coast         54
Ipswich                47
Logan - Beaudesert     44
Brisbane - East        33
Brisbane - West        26
Brisbane - South       25
Brisbane Inner City    23
Moreton Bay - South    22
Brisbane - North       17
Moreton Bay - North    13
Name: HH_SA4, dtype: int64

In [17]:
"""Calculate Primary kids."""
trip_rev = TRIPS.copy()
school_pers = Person.copy()
school_pers['STUDYING'].replace({'other': 0}, inplace = True)

"""Age Bands."""
under6 = school_pers.AGE < 5
over5 = school_pers.AGE > 4
under13 = school_pers.AGE < 13
"""Processing."""
duration = trip_rev.DURATION > 120
zero_study = school_pers.STUDYING == 0
DEST_prim= trip_rev.DESTPLACE == 'Primary or secondary school'

prim_person = trip_rev.loc[DEST_prim & duration,['PERSID']].drop_duplicates().astype({'PERSID': 'int64'})
prim_person['Prim_person'] = 1
school_pers = pd.merge(school_pers, prim_person, on='PERSID', how='left')
missing = school_pers.STUDYING == 0

prim_missing = school_pers.Prim_person == 1

school_pers.loc[missing & prim_missing & over5 & under13, 'STUDYING'] = 'Primary'
school_pers.loc[missing & prim_missing & under6, 'STUDYING'] = 'CC/Kindy'
print("Number of Added Kindy Kids:{}".format(len(school_pers.loc[missing & prim_missing & under6])))
print("Number of Added Primary Kids:{}".format(len(school_pers.loc[missing & prim_missing & over5 & under13])))

school_pers.drop(columns = 'Prim_person', inplace = True)

Person = school_pers

del school_pers, prim_person, prim_missing, DEST_prim, under6, over5, under13, trip_rev, missing

Number of Added Kindy Kids:49
Number of Added Primary Kids:24


In [18]:
pers_filt = Person.copy()

"""Age Bands."""
over5 = pers_filt.AGE > 4
under5 = pers_filt.AGE < 5
under18 = pers_filt.AGE < 18
over15 = pers_filt.AGE > 14
sub75 = pers_filt.AGE < 75
over75 = pers_filt.AGE > 74
under15 = pers_filt.AGE < 15
over20 = pers_filt.AGE > 20
o13 = pers_filt.AGE > 12
u13 = pers_filt.AGE < 12

"""Clean Studying Var."""
PrimSchl = pers_filt.STUDYING == 'Primary'
SecSchl = pers_filt.STUDYING == 'Secondary'
UniFT = pers_filt.STUDYING == 'tertiaryFullTime'
UniPT = pers_filt.STUDYING == 'tertiaryPartTime'
CC_kindy = pers_filt.MAINACT == 'Childcare/Kindergarten'

# print('CC/Kindy')
# print(pers_filt.loc[CC_kindy,['STUDYING','AGE']].value_counts())
pers_filt.loc[CC_kindy,'STUDYING'] = "CC/Kindy"
# print("Primary to Kindy")
# print(pers_filt.loc[PrimSchl & under5, ['STUDYING','AGE']].value_counts())
pers_filt.loc[PrimSchl & under5,'STUDYING'] = "CC/Kindy"
# print("Primary to Secondary")
# print(pers_filt.loc[PrimSchl & o13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[PrimSchl & o13,'STUDYING'] = 'Secondary'
# print("Adult Secondary")
# print(pers_filt.loc[SecSchl & over20, ['STUDYING','AGE']].value_counts())
pers_filt.loc[SecSchl & over20,'STUDYING'] = 0
# print("Secondary to Primary")
# print(pers_filt.loc[SecSchl & u13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[SecSchl & u13,'STUDYING'] = 'Primary'
# print('Uni errors')
# print(pers_filt.loc[UniFT & u13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[UniFT & u13,'STUDYING'] = 0
# print(pers_filt.loc[UniPT & u13, ['STUDYING','AGE']].value_counts())
pers_filt.loc[UniPT & u13,'STUDYING'] = 0

#fig = px.histogram(pers_filt[['STUDYING',"MAINACT",'AGE']], x="AGE", color="STUDYING", marginal = "rug", hover_data=['MAINACT']);fig.show()
"""Assign Study Vars."""
PrimSchl = pers_filt.STUDYING == 'Primary'
SecSchl = pers_filt.STUDYING == 'Secondary'
TertPT = pers_filt.STUDYING == 'tertiaryFullTime'
TertFT = pers_filt.STUDYING == 'tertiaryPartTime'

pers_filt.loc[PrimSchl, 'Student_Prim'] = 1
pers_filt.loc[SecSchl, 'Student_Sec'] = 1
pers_filt.loc[TertPT | TertFT, 'Student_Tert'] = 1
pers_filt.loc[PrimSchl | SecSchl | TertPT | TertFT, 'Student'] = 1

"""Clean Worker Variable."""
retired = pers_filt.WORKSTATUS == 'retired'
PrimSchl = pers_filt.STUDYING == 'Primary'

#display(pers_filt.loc[PrimSchl| retired | under15, ['Collar','AGE']].plot.hist(bins=20))
pers_filt.loc[PrimSchl| retired | under15,'Collar'] = np.NaN

white = pers_filt.Collar == 'White'
blue = pers_filt.Collar == 'Blue'

"""Assign Worker vs Dependents."""
workFT = pers_filt.WORKSTATUS == 'workFullTime'
workPT = pers_filt.WORKSTATUS == 'workPartTime'
non_workFT = pers_filt.WORKSTATUS != 'workFullTime'
non_workPT = pers_filt.WORKSTATUS != 'workPartTime'

pers_filt.loc[non_workFT & non_workPT, 'Dependent'] = 1
pers_filt.loc[workFT | workPT, 'Worker'] = 1
worker = pers_filt.Worker == 1

pers_filt.loc[worker & white, 'Worker_White'] = 1
pers_filt.loc[worker & blue, 'Worker_Blue'] = 1

"""Apply Age Categories."""
u15 = pers_filt.AGE < 15
o15 = pers_filt.AGE > 14

u30 = pers_filt.AGE < 30
o30 = pers_filt.AGE > 29

u50 = pers_filt.AGE < 50
o50 = pers_filt.AGE > 49

u75 = pers_filt.AGE < 75
o75 = pers_filt.AGE > 74

pers_filt.loc[u15, '0-14'] = 1
pers_filt.loc[o15 & u30, '15-29'] = 1
pers_filt.loc[o30 & u50, '30-49'] = 1
pers_filt.loc[o50 & u75, '50-74'] = 1
pers_filt.loc[o75, '75+'] = 1

pers_filt.loc[u15,'Age_cat'] = '0-14'
pers_filt.loc[o15 & u30,'Age_cat'] =  '15-29'
pers_filt.loc[o30 & u50,'Age_cat'] =  '30-49'
pers_filt.loc[o50 & u75,'Age_cat'] =  '50-74'
pers_filt.loc[o75,'Age_cat'] =  '75+'


#Missing Record Check
pers_filt.fillna(0, inplace = True)
pers_filt['check'] = pers_filt['Worker'] + pers_filt['Dependent']
print('ERRORS PRESENT:{} Person Records without Variable attached'.format(pers_filt[pers_filt.check!=1].PERSID.count()))
print('REVIEW PERSID:{} '.format(pers_filt[pers_filt.check!=1].PERSID.to_list()))
pers_filt.drop(columns = ['check'], inplace = True)

Person = pers_filt
del PrimSchl, SecSchl, TertPT, TertFT
del workFT, workPT, non_workFT, non_workPT, white, blue
del u15, o15,u30, o30, u50, o50, u75, o75
del pers_filt
del over5, under5, under18, over15, sub75, over75, under15, over20, o13, u13
del retired, worker, UniFT, UniPT

ERRORS PRESENT:0 Person Records without Variable attached
REVIEW PERSID:[] 


In [19]:
"""Calculate Workers."""
trip_rev = TRIPS.copy()
work_pers = Person.copy()

"""Age Bands."""
over15 = work_pers.AGE > 14

"""Processing."""
duration = trip_rev.DURATION > 60 #--------was 60-----------------------------------------------------------------------------------
DEST_WP = trip_rev.DESTPLACE == 'My workplace'

work_person = trip_rev[['PERSID',"HHID", "DESTPLACE", "DURATION"]].loc[duration & DEST_WP,['PERSID']].drop_duplicates().astype({'PERSID': 'int64'})

work_person['does_work'] = 1

work_pers = pd.merge(work_pers, work_person, on='PERSID', how='left')
work_person_type = work_pers['does_work'] == 1
not_worker = work_pers.Worker != 1
blue_col = work_pers.Collar == 'Blue'
white_col = work_pers.Collar == 'White'
no_col = work_pers.Collar == 0
work_pers.loc[work_person_type & over15 & not_worker,'Worker'] = 1
work_pers.loc[work_person_type & over15 & not_worker & blue_col,'Worker_Blue'] = 1
work_pers.loc[work_person_type & over15 & not_worker & white_col,'Worker_White'] = 1
work_pers.loc[work_person_type & over15 & not_worker & no_col,'Worker_White'] = 1
print("Number of Added Workers:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker])))
print("Number of Added Workers - White:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker & white_col])))
print("Number of Added Workers - Blue:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker & blue_col])))
print("Number of Added Workers - None:{}".format(len(work_pers.loc[work_person_type & over15 & not_worker & no_col])))
work_pers.drop(columns=['does_work'], inplace = True)

Person = work_pers
del work_person, trip_rev, over15, blue_col, white_col, no_col, work_pers, work_person_type, not_worker, DEST_WP, duration

Number of Added Workers:291
Number of Added Workers - White:6
Number of Added Workers - Blue:1
Number of Added Workers - None:284


In [20]:
grouped_person = Person[['PERSID','0-14', '15-29', '30-49','50-74', '75+','Age_cat', 'Dependent', 'Worker', 'Student','Worker_Blue', 'Worker_White', 'Student_Prim', 'Student_Sec','Student_Tert','Student_CC', 'AGE','HHID']].groupby('HHID').agg({'PERSID': 'count','0-14': 'sum', '15-29': 'sum', '30-49': 'sum','50-74': 'sum', '75+': 'sum', 'Worker_Blue':'sum', 'Worker_White': 'sum','Worker': 'sum', 'Student_Prim': 'sum', 'Student_Sec': 'sum','Student_Tert': 'sum','Student_CC': 'sum','Student': 'sum', 'AGE': 'mean'}).rename(columns={'PERSID':'Residents'})
Household = pd.merge(Household, grouped_person, left_on = 'HHID', right_index = True)
del grouped_person

In [21]:
trip = TRIPS.copy()
trip = trip.astype({'TRIPID': 'int64', 'HHID':'int64', "PERSID": 'int64'})
trip['trip_stops'] = trip['ENDSTOP'] - trip['STARTSTOP']
#trip = pd.merge(trip, Person[['PERSID','Collar', 'AGE','HHID']], left_on = 'PERSID', right_on = 'PERSID',how='left').drop(columns='HHID_y').rename(columns={'HHID_x': 'HHID'})
trip = pd.merge(trip, Person[['PERSID','Collar','Collar2','WORKSTATUS','0-14', '15-29', '30-49','50-74', '75+','Age_cat', 'Dependent', 'Worker', 'Student','Worker_Blue', 'Worker_White', 'Student_Prim', 'Student_Sec','Student_Tert', 'Student_CC','AGE','HHID', 'HH_SA2', 'HH_SA3', 'HH_SA4', 'HH_LGA','Home_LvL4', 'Home_LvL3', 'Home_LvL2', 'Home_LvL1', 'Home_Dist', 'BSTM']], left_on = 'PERSID', right_on = 'PERSID', how='left').drop(columns='HHID_y').rename(columns={'HHID_x': 'HHID','SA2_NAME_2016': "HH_SA2", 'SA3_NAME_2016': "HH_SA3", 'SA4_NAME_2016': "HH_SA4", 'LGA_NAME_2016': "HH_LGA", 'BSTM': 'HH_BSTM'})

#trip.rename(columns={"TRIPWGT19":'old_TRIPWGT19'},inplace=True)#reweight##############
#trip = pd.merge(trip, Pers.rename(columns={'PERSWGT19':'TRIPWGT19'}), on='PERSID', how='left')#reweight##############

print('Number of Trip Records in Original: {}'.format(len(TRIPS.axes[0])))
print('Number of Trip Records: {}'.format(len(trip.axes[0])))

trip['old_TRIPWGT19'] = trip['TRIPWGT19']
trip = trip.set_index("PERSID")
TripPers = Pers.rename(columns={'PERSWGT19':'TRIPWGT19'})
trip.update(TripPers.set_index('PERSID'))
trip.reset_index(inplace=True)
del TripPers

Number of Trip Records in Original: 101642
Number of Trip Records: 101642


In [170]:
"""Purpose Adjustments."""
purp_filt = trip.copy()
dur_u15 = purp_filt.DURATION < 16
dur_o15 = purp_filt.DURATION > 15
dur_o60 = purp_filt.DURATION > 59
dur_o180 = purp_filt.DURATION > 179

purp_filt.loc[dur_u15, 'Dur'] = 'u15'
purp_filt.loc[dur_o15, 'Dur'] = 'o15'
purp_filt.loc[dur_o60, 'Dur'] = 'o60'
purp_filt.loc[dur_o180, 'Dur'] = 'o180'
purp_filt.head()

,PERSID,TRIPID,HHID,STARTSTOP,ENDSTOP,STARTIME,ORIGPLACE,ORIGPURP,ORIGLONG,ORIGLAT,...,ATTR_SA4,ATTR_LGA,ORIG_SA2,ORIG_SA3,ORIG_SA4,ORIG_LGA,DEST_SA2,DEST_SA3,DEST_SA4,DEST_LGA
0,11000,11000001,1,1,1,255,My home,At Home,153.020211,-27.207136,...,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R)
1,11000,11000002,1,2,2,400,Shop or supermarket,Shopping,153.016427,-27.242199,...,Moreton Bay - South,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)
2,11001,11001001,1,1,1,255,My home,At Home,153.020211,-27.207136,...,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R)
3,11001,11001002,1,2,2,400,Shop or supermarket,Shopping,153.016427,-27.242199,...,Moreton Bay - South,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)
4,11001,11001003,1,3,3,430,My home,Go home,153.020211,-27.207136,...,Moreton Bay - North,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)


In [171]:
""" Home Based
    Logic:
    Place: 'My Home' = Home Based trip
    ORIGPLACE: 'My Home' -> P2A Trip -> HB_Orig Flag
    DESTPLACE: 'My Home' -> A2P Trip -> HB_Dest Flag
    Flags:
    HB_Orig or HB_Dest = 'Home_based' Flag
"""
home_is_orig = purp_filt.ORIGPLACE == 'My home'
purp_filt.loc[home_is_orig, 'HB_Orig'] = 1
home_is_dest = purp_filt.DESTPLACE == 'My home'
purp_filt.loc[home_is_dest, 'HB_Dest'] = 1
purp_filt.loc[home_is_orig | home_is_dest, 'Home_based'] = 1


purp_filt.loc[home_is_orig, 'Orig_Type'] = 'Home'
purp_filt.loc[home_is_dest, 'Dest_Type'] = 'Home'

home_purp_is_orig = purp_filt.ORIGPURP == 'At Home'
purp_filt.loc[home_purp_is_orig, 'HB_Orig'] = 1
home_purp_is_dest = purp_filt.DESTPURP == 'Go home'
purp_filt.loc[home_purp_is_dest, 'HB_Dest'] = 1
purp_filt.loc[home_purp_is_orig | home_purp_is_dest, 'Home_based'] = 1

del home_is_orig, home_is_dest, home_purp_is_orig, home_purp_is_dest
purp_filt.head()

,PERSID,TRIPID,HHID,STARTSTOP,ENDSTOP,STARTIME,ORIGPLACE,ORIGPURP,ORIGLONG,ORIGLAT,...,ATTR_SA4,ATTR_LGA,ORIG_SA2,ORIG_SA3,ORIG_SA4,ORIG_LGA,DEST_SA2,DEST_SA3,DEST_SA4,DEST_LGA
0,11000,11000001,1,1,1,255,My home,At Home,153.020211,-27.207136,...,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R)
1,11000,11000002,1,2,2,400,Shop or supermarket,Shopping,153.016427,-27.242199,...,Moreton Bay - South,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)
2,11001,11001001,1,1,1,255,My home,At Home,153.020211,-27.207136,...,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R)
3,11001,11001002,1,2,2,400,Shop or supermarket,Shopping,153.016427,-27.242199,...,Moreton Bay - South,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)
4,11001,11001003,1,3,3,430,My home,Go home,153.020211,-27.207136,...,Moreton Bay - North,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)


In [172]:
""" Work Trip
    Logic:
    PLACE: 'My Workplace' = Work Trip
    PLACE: 'Other work-related place' = Work Trip
    Flags:
    ORIG/DEST Workplace or other Related = 'work' Flag
"""
work_orig = purp_filt.ORIGPLACE == 'My workplace' 
work_dest = purp_filt.DESTPLACE == 'My workplace'
work_orig_other = purp_filt.ORIGPLACE == 'Other work-related place'
work_dest_other = purp_filt.DESTPLACE == 'Other work-related place'

# purp_filt.loc[work_orig | work_dest | work_orig_other | work_dest_other, 'work'] = 1
# purp_filt.loc[work_orig | work_orig_other, 'Orig_Type'] = 'work'
# purp_filt.loc[work_dest | work_dest_other, 'Dest_Type'] = 'work'

## Atousa Modification: last three lines are being commented since the work flag should be True if the purpose is "work" not 
## the places. In some record, even though the origplace or destplace is work-related, the purpose for the trip is not working.

del work_orig, work_dest, work_dest_other, work_orig_other


#Work Purpose
work_o_purp = purp_filt.ORIGPURP == 'Work (my workplace)'
work_d_purp = purp_filt.DESTPURP == 'Work (my workplace)'
work_o_purp2 = purp_filt.ORIGPURP == 'Work (other work reason)'
work_d_purp2 = purp_filt.DESTPURP == 'Work (other work reason)'

## Atousa Modification: The nest three lines are added
purp_filt.loc[work_o_purp | work_d_purp | work_o_purp2 | work_d_purp2, 'work'] = 1
purp_filt.loc[work_o_purp | work_o_purp2, 'Orig_Type'] = 'work'
purp_filt.loc[work_d_purp | work_d_purp2, 'Dest_Type'] = 'work'

del work_o_purp, work_d_purp, work_o_purp2, work_d_purp2
purp_filt.head()

,PERSID,TRIPID,HHID,STARTSTOP,ENDSTOP,STARTIME,ORIGPLACE,ORIGPURP,ORIGLONG,ORIGLAT,...,ATTR_SA4,ATTR_LGA,ORIG_SA2,ORIG_SA3,ORIG_SA4,ORIG_LGA,DEST_SA2,DEST_SA3,DEST_SA4,DEST_LGA
0,11000,11000001,1,1,1,255,My home,At Home,153.020211,-27.207136,...,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R)
1,11000,11000002,1,2,2,400,Shop or supermarket,Shopping,153.016427,-27.242199,...,Moreton Bay - South,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)
2,11001,11001001,1,1,1,255,My home,At Home,153.020211,-27.207136,...,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R)
3,11001,11001002,1,2,2,400,Shop or supermarket,Shopping,153.016427,-27.242199,...,Moreton Bay - South,Moreton Bay (R),North Lakes - Mango Hill,North Lakes,Moreton Bay - South,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)
4,11001,11001003,1,3,3,430,My home,Go home,153.020211,-27.207136,...,Moreton Bay - North,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R),Deception Bay,Narangba - Burpengary,Moreton Bay - North,Moreton Bay (R)


In [173]:
##Education Flag
#Purpose
educ_purp_o = purp_filt.ORIGPURP == 'Education'
educ_purp_d = purp_filt.DESTPURP == 'Education'

purp_filt.loc[educ_purp_o | educ_purp_d, 'Edu_purp'] = 1

""" Primary/Secondary
    Logic: 
    Place: 'Primary or secondary school' = HBE Trip
    Place: 'Childcare or kindergarten' = HBE Trip
    Flags:
    ORIG/DEST Prim/Sec or Child/Kindy = "Edu_p/s" Flag
"""
prim = purp_filt.Student_Prim == 1
sec = purp_filt.Student_Sec == 1
tert = purp_filt.Student_Tert == 1
CC_kids = purp_filt.Student_CC == 1
non_student = purp_filt.Student != 1
purp_filt['Student_ALL'] = purp_filt['Student_Prim'] + purp_filt['Student_Sec'] + purp_filt['Student_Tert'] + purp_filt['Student_CC']
non_all_student = purp_filt.Student_ALL == 0
purp_filt['Student_Young'] = purp_filt['Student_Prim'] + purp_filt['Student_Sec'] + purp_filt['Student_CC']
non_young_student = purp_filt.Student_Young == 0

primsec_orig = purp_filt.ORIGPLACE == 'Primary or secondary school' 
primsec_dest = purp_filt.DESTPLACE == 'Primary or secondary school' 
CCKindy_orig = purp_filt.ORIGPLACE == 'Childcare or kindergarten' 
CCKindy_dest = purp_filt.DESTPLACE == 'Childcare or kindergarten'
other_ed_orig = purp_filt.ORIGPLACE == 'Other study-related place' 
other_ed_dest = purp_filt.DESTPLACE == 'Other study-related place'
campus_ed_orig = purp_filt.ORIGPLACE == 'My campus' 
campus_ed_dest = purp_filt.DESTPLACE == 'My campus'

purp_filt.loc[primsec_orig | primsec_dest | CCKindy_orig | CCKindy_dest | other_ed_orig | other_ed_dest | campus_ed_orig | campus_ed_dest, 'Edu_place'] = 1

purp_filt.loc[primsec_orig | primsec_dest | CCKindy_orig | CCKindy_dest | other_ed_orig | other_ed_dest, 'Edu_cps_place'] = 1

purp_filt.loc[primsec_orig | CCKindy_orig | other_ed_orig | campus_ed_orig , 'Orig_Type'] = 'Education'
purp_filt.loc[primsec_dest | CCKindy_dest | other_ed_dest | campus_ed_dest, 'Dest_Type'] = 'Education'

"""
Categorise education trips by students
"""
educ_O_D = purp_filt.Edu_place == 1
purp_filt.loc[primsec_orig & prim, 'Edu_p'] = 1
purp_filt.loc[primsec_dest & prim, 'Edu_p'] = 1
purp_filt.loc[other_ed_orig & prim, 'Edu_p'] = 1
purp_filt.loc[other_ed_dest & prim, 'Edu_p'] = 1
purp_filt.loc[campus_ed_orig & prim, 'Edu_p'] = 1
purp_filt.loc[campus_ed_dest & prim, 'Edu_p'] = 1

purp_filt.loc[primsec_orig & sec, 'Edu_s'] = 1
purp_filt.loc[primsec_dest & sec, 'Edu_s'] = 1
purp_filt.loc[other_ed_orig & sec, 'Edu_s'] = 1
purp_filt.loc[other_ed_dest & sec, 'Edu_s'] = 1
purp_filt.loc[campus_ed_orig & sec, 'Edu_s'] = 1
purp_filt.loc[campus_ed_dest & sec, 'Edu_s'] = 1

purp_filt.loc[other_ed_orig & tert, 'Edu_t'] = 1
purp_filt.loc[other_ed_dest & tert, 'Edu_t'] = 1
purp_filt.loc[campus_ed_orig & tert, 'Edu_t'] = 1
purp_filt.loc[campus_ed_dest & tert, 'Edu_t'] = 1

purp_filt.loc[CCKindy_orig & CC_kids, 'Edu_c'] = 1
purp_filt.loc[CCKindy_dest & CC_kids, 'Edu_c'] = 1

purp_filt.loc[educ_O_D & non_all_student, 'Edu_non'] = 1

"""Shopping."""
shop_orig = purp_filt.ORIGPLACE == 'Shop or supermarket' 
shop_dest = purp_filt.DESTPLACE == 'Shop or supermarket'
food_orig = purp_filt.ORIGPLACE == 'Restaurant, cafÃ©, or food outlet'
food_dest = purp_filt.DESTPLACE == 'Restaurant, cafÃ©, or food outlet'
fuel_orig = purp_filt.ORIGPLACE == 'Petrol station'
fuel_dest = purp_filt.DESTPLACE == 'Petrol station'

purp_filt.loc[shop_orig | shop_dest | food_orig | food_dest | fuel_dest | fuel_orig , 'shop'] = 1
purp_filt.loc[shop_orig | food_orig, 'Orig_Type'] = 'Shopping'
purp_filt.loc[shop_dest | food_dest, 'Dest_Type'] = 'Shopping'
purp_filt.loc[fuel_orig , 'Orig_Type'] = 'Fuel'
purp_filt.loc[fuel_dest , 'Dest_Type'] = 'Fuel'
del shop_orig, shop_dest, food_orig, food_dest, fuel_dest, fuel_orig

#Escort Flag
escort_purp_orig_1_flag = purp_filt.ORIGPURP == 'Transporting a passenger/s'
escort_purp_dest_1_flag = purp_filt.DESTPURP == 'Transporting a passenger/s'
escort_purp_orig_2_flag = purp_filt.ORIGPURP == 'Pick up or drop off a passenger'
escort_purp_dest_2_flag = purp_filt.DESTPURP == 'Pick up or drop off a passenger'
escort_purp_orig_3_flag = purp_filt.ORIGPURP == 'Just accompanying someone'
escort_purp_dest_3_flag = purp_filt.DESTPURP == 'Just accompanying someone'

purp_filt.loc[escort_purp_orig_1_flag | escort_purp_orig_2_flag | escort_purp_dest_1_flag | escort_purp_dest_2_flag | escort_purp_orig_3_flag | escort_purp_dest_3_flag, 'escort'] = 1

del escort_purp_orig_1_flag, escort_purp_orig_2_flag, escort_purp_dest_1_flag, escort_purp_dest_2_flag
del escort_purp_orig_3_flag, escort_purp_dest_3_flag

#Other Flag
other_home_orig = purp_filt.ORIGPLACE == "Someone else's home"
other_home_dest = purp_filt.DESTPLACE == "Someone else's home"
purp_filt.loc[other_home_orig | other_home_dest, 'others home'] = 1
other_place_orig = purp_filt.ORIGPLACE == "Other place (please specify)"
other_place_dest = purp_filt.DESTPLACE == "Other place (please specify)"
purp_filt.loc[other_home_orig | other_home_dest | other_place_orig | other_place_dest, 'other'] = 1
purp_filt.loc[other_home_orig, 'Orig_Type'] = 'Other Home'
purp_filt.loc[other_place_orig , 'Orig_Type'] = 'Other Place'
purp_filt.loc[other_home_dest, 'Dest_Type'] = 'Other Home'
purp_filt.loc[other_place_dest, 'Dest_Type'] = 'Other Place'
del other_home_orig, other_home_dest, other_place_orig, other_place_dest

cp_orig = purp_filt.ORIGPLACE == 'Car park'
ferry_orig = purp_filt.ORIGPLACE == 'Ferry terminal'
lr_orig = purp_filt.ORIGPLACE == 'Light rail stop'
pb_orig = purp_filt.ORIGPLACE == 'Public bus stop'
sb_orig = purp_filt.ORIGPLACE == 'School bus stop'
ts_orig = purp_filt.ORIGPLACE == 'Train station'
purp_filt.loc[cp_orig | ferry_orig | lr_orig | pb_orig | sb_orig | ts_orig, 'Orig_Type'] = 'PT Station'

cp_dest= purp_filt.DESTPLACE == 'Car park'
ferry_dest = purp_filt.DESTPLACE == 'Ferry terminal'
lr_dest = purp_filt.DESTPLACE == 'Light rail stop'
pb_dest = purp_filt.DESTPLACE == 'Public bus stop'
sb_dest = purp_filt.DESTPLACE == 'School bus stop'
ts_dest = purp_filt.DESTPLACE == 'Train station'
purp_filt.loc[cp_dest | ferry_dest | lr_dest | pb_dest | sb_dest | ts_dest, 'Dest_Type'] = 'PT Station'

purp_filt.fillna(0, inplace = True)

In [174]:
"""Trip Purpose Calculator"""
purp_filt['Trip_Purpose'] = 0

#HBW - White & Blue
w_collar_flag = purp_filt.Worker_White == 1
b_collar_flag = purp_filt.Worker_Blue == 1
hb_flag = purp_filt.Home_based == 1
work_flag = purp_filt.work == 1
purp_filt.loc[w_collar_flag & hb_flag & work_flag, 'Trip_Purpose'] = 'HBWW'
done_flag = purp_filt.Trip_Purpose == 0
purp_filt.loc[b_collar_flag & hb_flag & work_flag, 'Trip_Purpose'] = 'HBWB'

del w_collar_flag, b_collar_flag, work_flag, hb_flag

done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
edu_purp_flag = purp_filt["Edu_purp"] == 1
edu_p_flag = purp_filt['Edu_p'] == 1
edu_s_flag = purp_filt['Edu_s'] == 1
edu_t_flag = purp_filt['Edu_t'] == 1
edu_c_flag = purp_filt['Edu_c'] == 1

purp_filt.loc[done_flag & hb_flag & edu_purp_flag & edu_p_flag, 'Trip_Purpose'] = 'HBEp'
purp_filt.loc[done_flag & hb_flag & edu_purp_flag & edu_s_flag, 'Trip_Purpose'] = 'HBEs'
purp_filt.loc[done_flag & hb_flag & edu_purp_flag & edu_t_flag, 'Trip_Purpose'] = 'HBEt'
purp_filt.loc[done_flag & hb_flag  & edu_c_flag, 'Trip_Purpose'] = 'HBEc'    
## Atousa Modification: edu_purp_flag should be added to the conditions for the HBEc catagory

In [175]:
purp_filt.Trip_Purpose.value_counts()

0       70305
HBWW    14672
HBWB     6263
HBEp     4929
HBEs     3994
HBEt      847
HBEc      632
Name: Trip_Purpose, dtype: int64

In [176]:
"""Reclassify Escorted School trips."""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
hb_o_flag = purp_filt.HB_Orig == 1
hb_d_flag = purp_filt.HB_Dest == 1
edu_cps_flag = purp_filt['Edu_cps_place'] == 1
escort_flag = purp_filt.escort == 1
CC_student = purp_filt.Student_CC == 1
prim_student = purp_filt.Student_Prim == 1
young_student = purp_filt.Student_Young == 1
dur_u15_a = purp_filt.Dur == 'u15'
dur_o60_a = purp_filt.Dur == 'o60'

purp_filt.loc[done_flag & hb_o_flag & edu_cps_flag & CC_student & dur_o60_a, 'Trip_Purpose'] = 'HBEc'
purp_filt.loc[done_flag & hb_o_flag & edu_cps_flag & prim_student & dur_o60_a, 'Trip_Purpose'] = 'HBEp'
purp_filt.loc[done_flag & hb_d_flag & edu_cps_flag & CC_student & dur_o60_a, 'Trip_Purpose'] = 'HBEc'
purp_filt.loc[done_flag & hb_d_flag & edu_cps_flag & prim_student & dur_o60_a, 'Trip_Purpose'] = 'HBEp'
## Atousa: why should we add "edu_cps_flag" as a condition? In fact, by considering this flag, primary/secondary 
## places will be considered for HBEc which is not rational. 
del CC_student, prim_student, young_student, dur_u15_a, dur_o60_a

In [177]:
purp_filt.Trip_Purpose.value_counts()

0       70236
HBWW    14672
HBWB     6263
HBEp     4995
HBEs     3994
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64

In [178]:
"""Education Escort Trips."""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
hb_o_flag = purp_filt.HB_Orig == 1
hb_d_flag = purp_filt.HB_Dest == 1
driver_flag = purp_filt.MAINMODE == 'Car driver'
escort_flag = purp_filt.escort == 1
non_student = purp_filt.Student_Young != 1#################################
young_student = purp_filt.Student_Young == 1
dur_u15_a = purp_filt.Dur == 'u15'

edu_cps_flag = purp_filt['Edu_cps_place'] == 1
purp_filt.loc[done_flag & hb_flag & edu_cps_flag & driver_flag & escort_flag, 'Trip_Purpose'] = 'HBEe' #
done_flag = purp_filt.Trip_Purpose == 0

# purp_filt.loc[done_flag & hb_flag & edu_cps_flag & driver_flag & edu_purp_flag , 'Trip_Purpose'] = 'HBEe'
## Atousa Modification: the last line does not consider the escort notion: someone may be car driver but with the purpose
## of education, thus this trip is HBE but not HBEe (so the line is commented)

purp_filt.loc[done_flag & hb_flag & edu_cps_flag & non_student & escort_flag , 'Trip_Purpose'] = 'HBEe'
purp_filt.loc[done_flag & hb_o_flag & edu_cps_flag & young_student & escort_flag & dur_u15_a , 'Trip_Purpose'] = 'HBEe'
purp_filt.loc[done_flag & hb_d_flag & edu_cps_flag & young_student & escort_flag, 'Trip_Purpose'] = 'HBEe'

## Atousa suggestion: If HBEe is defined as a kind of trip with escorting purpose (between home and education place),
## it is better to just consider the escort flag, hb_flag and education places flag (considering all types of education places
## including campus instead of just considering CCkindy, prim/sec and other places). Also by using this notion, the last
## three lines are not applicable.

del non_student, young_student, dur_u15_a, escort_flag, hb_o_flag, hb_d_flag,hb_flag

In [179]:
purp_filt.Trip_Purpose.value_counts()

0       60109
HBWW    14672
HBEe    10127
HBWB     6263
HBEp     4995
HBEs     3994
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64

In [180]:
"""HBS - Shopping"""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
shop_flag = purp_filt.shop == 1

# purp_filt.loc[done_flag & hb_flag & shop_flag, 'Trip_Purpose'] = 'HBS' 
# del done_flag, hb_flag, shop_flag
## Atousa Modification: the last two lines become comment since they check the places not the purposes for HBS trips. The next
## lines are added.

eat_flag_orig = purp_filt.ORIGPURP == 'Eat/drink'
eat_flag_dest = purp_filt.DESTPURP == 'Eat/drink'
shop_flag_orig = purp_filt.ORIGPURP == 'Shopping'
shop_flag_dest = purp_filt.DESTPURP == 'Shopping'
fuel_flag_orig = purp_filt.ORIGPURP == 'Refuel my vehicle'
fuel_flag_dest = purp_filt.DESTPURP == 'Refuel my vehicle'

purp_filt.loc[eat_flag_orig | eat_flag_dest | shop_flag_orig | shop_flag_dest | fuel_flag_orig | fuel_flag_dest, 'shop_purp'] = 1
shop_purp_flag = purp_filt.shop_purp == 1
purp_filt.loc[shop_purp_flag & shop_flag & hb_flag & done_flag, "Trip_Purpose"] = 'HBS'


In [181]:
purp_filt.Trip_Purpose.value_counts()

0       46903
HBWW    14672
HBS     13206
HBEe    10127
HBWB     6263
HBEp     4995
HBEs     3994
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64

In [182]:
"""#HBO Recreation"""
rec_orig = purp_filt.ORIGPURP == 'Recreation/leisure activity'
rec_dest = purp_filt.DESTPURP == 'Recreation/leisure activity'
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1

purp_filt.loc[done_flag & hb_flag & rec_orig, 'Trip_Purpose'] = 'HBOr'
purp_filt.loc[done_flag & hb_flag & rec_dest, 'Trip_Purpose'] = 'HBOr'
del rec_orig, rec_dest, hb_flag, done_flag

In [183]:
purp_filt.Trip_Purpose.value_counts()

0       36557
HBWW    14672
HBS     13206
HBOr    10346
HBEe    10127
HBWB     6263
HBEp     4995
HBEs     3994
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64

In [184]:
"""HBO - Escort & Other"""
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
escort_flag = purp_filt.escort == 1
other_flag = purp_filt.other == 1
purp_filt.loc[done_flag & hb_flag & other_flag, 'Trip_Purpose'] = 'HBO'
purp_filt.loc[done_flag & hb_flag & other_flag & escort_flag, 'Trip_Purpose'] = 'HBOe'
del done_flag, hb_flag, escort_flag, other_flag

In [185]:
purp_filt.Trip_Purpose.value_counts()

0       23741
HBWW    14672
HBS     13206
HBOr    10346
HBEe    10127
HBO      8740
HBWB     6263
HBEp     4995
HBOe     4076
HBEs     3994
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64

In [186]:
#NHB - Education, Work, Shop and Other
done_flag = purp_filt.Trip_Purpose == 0
non_hb_flag = purp_filt.Home_based != 1

#Work Purpose
work_d_purp = purp_filt.DESTPURP == 'Work (my workplace)'
work_d_purp2 = purp_filt.DESTPURP == 'Work (other work reason)'
work_o_purp = purp_filt.ORIGPURP == 'Work (my workplace)'
work_o_purp2 = purp_filt.ORIGPURP == 'Work (other work reason)'
purp_filt.loc[work_d_purp | work_d_purp2, 'work_d'] = 1
work_d_flag = purp_filt.work_d == 1
purp_filt.loc[done_flag & non_hb_flag & work_d_flag, 'Trip_Purpose'] = 'NHBW'
## Atousa Modification: I don't understand why just destination with work purpose are considered but not origin.

#WorkTrip Directionality

purp_filt.loc[work_o_purp | work_o_purp2, 'work_o'] = 1
work_o_flag = purp_filt.work_o == 1
purp_filt.loc[non_hb_flag & work_o_flag, 'Work_Direction'] = 'Work2Attr'
purp_filt.loc[non_hb_flag & work_d_flag, 'Work_Direction'] = 'Attr2Work'
purp_filt.loc[done_flag & non_hb_flag & work_d_flag & work_o_flag, 'Trip_Purpose'] = 'WBW'

del work_d_purp, work_d_purp2, work_d_flag, work_o_purp, work_o_purp2, work_o_flag

In [187]:
purp_filt.Trip_Purpose.value_counts()

0       19441
HBWW    14672
HBS     13206
HBOr    10346
HBEe    10127
HBO      8740
HBWB     6263
HBEp     4995
HBOe     4076
HBEs     3994
NHBW     2151
WBW      2149
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64

In [188]:
#education NHB
done_flag = purp_filt.Trip_Purpose == 0
educ_purp_d = purp_filt.DESTPURP == 'Education'
primsec_dest = purp_filt.DESTPLACE == 'Primary or secondary school' 
CCKindy_dest = purp_filt.DESTPLACE == 'Childcare or kindergarten'
tert_dest = purp_filt.DESTPLACE == 'My campus' 
other_ed_dest = purp_filt.DESTPLACE == 'Other study-related place' 
tert_student = purp_filt.Student_Tert == 1
prim_student = purp_filt.Student_Prim == 1
sec_student = purp_filt.Student_Sec == 1 

pickdrop_1 = purp_filt.DESTPURP == 'Pick up or drop off a passenger'
pickdrop_2 = purp_filt.DESTPURP == 'Transporting a passenger/s'
pickdrop_3 = purp_filt.DESTPURP == 'Just accompanying someone'
edu_dest_flagger = purp_filt.Dest_Type == 'Education'

purp_filt.loc[primsec_dest | CCKindy_dest | tert_dest | other_ed_dest, 'edu_dest'] = 1
edu_dest_flag = purp_filt['edu_dest'] == 1

purp_filt.loc[done_flag & non_hb_flag & educ_purp_d & edu_dest_flag & prim_student, 'Trip_Purpose'] = 'NHBE'
purp_filt.loc[done_flag & non_hb_flag & educ_purp_d & edu_dest_flag & sec_student, 'Trip_Purpose'] = 'NHBE'

purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & pickdrop_1, 'Trip_Purpose'] = 'NHBE'
purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & pickdrop_2, 'Trip_Purpose'] = 'NHBE'
purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & pickdrop_3, 'Trip_Purpose'] = 'NHBE'
## Atousa Modification: the pickdrop_1,2,3 represent escort flag for destination. Also, I don't understand why origin
## escort flag and edu_origin_flagger is not considered. If origins get considered too, we can write the next two lines instead of the last 
## three lines.

# escort_flag = purp_filt.escort == 1
# purp_filt.loc[done_flag & non_hb_flag & edu_dest_flagger & escort_flag, 'Trip_Purpose'] = 'NHBE'

purp_filt.loc[done_flag & non_hb_flag & educ_purp_d & edu_dest_flag & tert_student, 'Trip_Purpose'] = 'NHBO' #Not NHBE

del done_flag, educ_purp_d, primsec_dest, CCKindy_dest, tert_dest, other_ed_dest, tert_student, prim_student, sec_student, edu_dest_flag

In [189]:
purp_filt.Trip_Purpose.value_counts()

0       16534
HBWW    14672
HBS     13206
HBOr    10346
HBEe    10127
HBO      8740
HBWB     6263
HBEp     4995
HBOe     4076
HBEs     3994
NHBE     2839
NHBW     2151
WBW      2149
HBEt      847
HBEc      635
NHBO       68
Name: Trip_Purpose, dtype: int64

In [190]:
#shopping
non_hb_flag = purp_filt.Home_based != 1
shop_dest = purp_filt.DESTPLACE == 'Shop or supermarket'
food_dest = purp_filt.DESTPLACE == 'Restaurant, cafÃ©, or food outlet'
fuel_dest = purp_filt.DESTPLACE == 'Petrol station'
purp_filt.loc[shop_dest | food_dest | fuel_dest, 'shop_dest'] = 1
done_flag = purp_filt.Trip_Purpose == 0
shop_d_flag = purp_filt.shop_dest == 1
purp_filt.loc[done_flag & non_hb_flag & shop_d_flag, 'Trip_Purpose'] = 'NHBS'
del done_flag, shop_d_flag, shop_dest, food_dest, fuel_dest

done_flag = purp_filt.Trip_Purpose == 0
other_flag = purp_filt.other == 1
purp_filt.loc[done_flag & non_hb_flag & other_flag, 'Trip_Purpose'] = 'NHBO'
del done_flag, non_hb_flag,  other_flag


#Non-Collared_Work
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
work_flag = purp_filt.work == 1



age_filter = purp_filt.AGE > 14
dest_accompy = purp_filt.DESTPURP != 'Just accompanying someone'
orig_accompy = purp_filt.ORIGPURP != 'Just accompanying someone'

dest_drop = purp_filt.DESTPURP != 'Pick up or drop off a passenger'
orig_drop = purp_filt.ORIGPURP != 'Pick up or drop off a passenger'

dest_rec = purp_filt.DESTPURP != 'Recreation/leisure activity'
orig_rec = purp_filt.ORIGPURP != 'Recreation/leisure activity'
dest_deliv = purp_filt.DESTPURP != 'Pickup/deliver something (not work-related)'
orig_deliv = purp_filt.ORIGPURP != 'Pickup/deliver something (not work-related)'
dest_social = purp_filt.DESTPURP != 'Social Visit'
orig_social = purp_filt.ORIGPURP != 'Social Visit'
dest_shop = purp_filt.DESTPURP != 'Shopping'
orig_shop = purp_filt.ORIGPURP != 'Shopping'


purp_filt.loc[done_flag & hb_flag & work_flag & age_filter & dest_accompy & orig_accompy & dest_drop & orig_drop & dest_rec & orig_rec & dest_deliv & orig_deliv & dest_social & orig_social & dest_shop & orig_shop, 'Trip_Purpose'] = 'HBWW'
#purp_filt.loc[done_flag & hb_flag & work_flag, 'Trip_Purpose'] = 'HBWW'

del age_filter, dest_accompy, orig_accompy, dest_drop, orig_drop, dest_rec, orig_rec, dest_deliv, orig_deliv, dest_social, orig_social, dest_shop, orig_shop

In [191]:
purp_filt.Trip_Purpose.value_counts()

HBWW    14859
HBS     13206
HBOr    10346
HBEe    10127
HBO      8740
HBWB     6263
0        5798
NHBS     5585
NHBO     5032
HBEp     4995
HBOe     4076
HBEs     3994
NHBE     2839
NHBW     2151
WBW      2149
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64

In [192]:
#HomeBaseHome LMAO xD
done_flag = purp_filt.Trip_Purpose == 0
home_orig = purp_filt.ORIGPLACE == 'My home'
home_dest = purp_filt.DESTPLACE == 'My home'
purp_filt.loc[done_flag & home_orig & home_dest, 'Trip_Purpose'] = 'HBOr' #now HBOr
del done_flag, home_orig, home_dest
#print(purp_filt['Trip_Purpose'].value_counts())

#Home Based Eating
## Atousa Modification: I commented this section since before I considered the relevant records for HBS
# done_flag = purp_filt.Trip_Purpose == 0
# home_flag = purp_filt.Home_based == 1
# eat_flag_orig = purp_filt.ORIGPURP == 'Eat/drink'
# eat_flag_dest = purp_filt.DESTPURP == 'Eat/drink'
# purp_filt.loc[eat_flag_orig | eat_flag_dest, 'eat_drink'] = 1
# eat_flag = purp_filt.eat_drink == 1
# purp_filt.loc[eat_flag & home_flag & done_flag, "Trip_Purpose"] = 'HBS'
# del done_flag, eat_flag_orig, eat_flag_dest, eat_flag, home_flag
# #print(purp_filt['Trip_Purpose'].value_counts())

#Home Based Escorting
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
escort_flag = purp_filt.escort == 1
purp_filt.loc[done_flag & hb_flag & escort_flag, 'Trip_Purpose'] = 'HBOe'
del done_flag, hb_flag, escort_flag


#print(purp_filt['Trip_Purpose'].value_counts())

#Home Based Other - Remainder
done_flag = purp_filt.Trip_Purpose == 0
hb_flag = purp_filt.Home_based == 1
purp_filt.loc[done_flag & hb_flag, 'Trip_Purpose'] = 'HBO'

del done_flag, hb_flag
#print(purp_filt['Trip_Purpose'].value_counts())
#Non-Home based Other - Remainder
done_flag = purp_filt.Trip_Purpose == 0
non_hb_flag = purp_filt.Home_based != 1
purp_filt.loc[done_flag & non_hb_flag, 'Trip_Purpose'] = 'NHBO'
del done_flag, non_hb_flag
print(purp_filt['Trip_Purpose'].value_counts())

trip = purp_filt
del purp_filt

HBWW    14859
HBS     13206
HBO     11238
HBOr    10370
HBEe    10127
HBOe     6857
HBWB     6263
NHBS     5585
NHBO     5527
HBEp     4995
HBEs     3994
NHBE     2839
NHBW     2151
WBW      2149
HBEt      847
HBEc      635
Name: Trip_Purpose, dtype: int64


In [120]:
"""Direction Flag."""
trip_dir = trip.copy()
trip_dir['Direction'] = 'OD'
p2a_flag = trip_dir.HB_Orig == 1
trip_dir.loc[p2a_flag, 'Direction'] = 'P2A'
a2p_flag = trip_dir.HB_Dest == 1
trip_dir.loc[a2p_flag, 'Direction'] = 'A2P'
del p2a_flag, a2p_flag

od_flag = trip_dir.Direction == 'OD'
trip_dir.loc[od_flag, 'PROD_SA1'] = trip_dir.loc[od_flag, 'ORIGSA1_2016'] 
trip_dir.loc[od_flag, 'ATTR_SA1'] = trip_dir.loc[od_flag, 'DESTSA1_2016']
trip_dir.loc[od_flag, 'PROD_LONG'] = trip_dir.loc[od_flag, 'ORIGLONG'] 
trip_dir.loc[od_flag, 'PROD_LAT'] = trip_dir.loc[od_flag, 'ORIGLAT'] 
trip_dir.loc[od_flag, 'ATTR_LONG'] = trip_dir.loc[od_flag, 'DESTLONG']
trip_dir.loc[od_flag, 'ATTR_LAT'] = trip_dir.loc[od_flag, 'DESTLAT'] 
del od_flag
p2a_flag = trip_dir.Direction == 'P2A'
trip_dir.loc[p2a_flag, 'PROD_SA1'] = trip_dir.loc[p2a_flag, 'ORIGSA1_2016'] 
trip_dir.loc[p2a_flag, 'ATTR_SA1'] = trip_dir.loc[p2a_flag, 'DESTSA1_2016'] 
trip_dir.loc[p2a_flag, 'PROD_LONG'] = trip_dir.loc[p2a_flag, 'ORIGLONG'] 
trip_dir.loc[p2a_flag, 'PROD_LAT'] = trip_dir.loc[p2a_flag, 'ORIGLAT'] 
trip_dir.loc[p2a_flag, 'ATTR_LONG'] = trip_dir.loc[p2a_flag, 'DESTLONG']
trip_dir.loc[p2a_flag, 'ATTR_LAT'] = trip_dir.loc[p2a_flag, 'DESTLAT'] 
del p2a_flag
a2p_flag = trip_dir.Direction == 'A2P'
trip_dir.loc[a2p_flag, 'PROD_SA1'] = trip_dir.loc[a2p_flag, 'DESTSA1_2016'] 
trip_dir.loc[a2p_flag, 'ATTR_SA1'] = trip_dir.loc[a2p_flag, 'ORIGSA1_2016'] 
trip_dir.loc[a2p_flag, 'PROD_LONG'] = trip_dir.loc[a2p_flag, 'DESTLONG'] 
trip_dir.loc[a2p_flag, 'PROD_LAT'] = trip_dir.loc[a2p_flag, 'DESTLAT'] 
trip_dir.loc[a2p_flag, 'ATTR_LONG'] = trip_dir.loc[a2p_flag, 'ORIGLONG']
trip_dir.loc[a2p_flag, 'ATTR_LAT'] = trip_dir.loc[a2p_flag, 'ORIGLAT'] 
del a2p_flag

trip = trip_dir
del trip_dir

"""Prod/Attr SA2-4,LGA Zones"""
zone_filt = trip.copy()
zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'PROD_SA1', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "PROD_SA2", 'SA3_NAME_2016': 'PROD_SA3', 'SA4_NAME_2016':'PROD_SA4', 'LGA_NAME_2016':'PROD_LGA'})
zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'ATTR_SA1', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "ATTR_SA2", 'SA3_NAME_2016': 'ATTR_SA3', 'SA4_NAME_2016':'ATTR_SA4', 'LGA_NAME_2016':'ATTR_LGA'})

zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'ORIGSA1_2016', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "ORIG_SA2", 'SA3_NAME_2016': 'ORIG_SA3', 'SA4_NAME_2016':'ORIG_SA4', 'LGA_NAME_2016':'ORIG_LGA'})
zone_filt = pd.merge(zone_filt, ASGS.set_index('SA1_7DIGITCODE_2016')[['SA2_NAME_2016','SA3_NAME_2016', 'SA4_NAME_2016','LGA_NAME_2016']], left_on = 'DESTSA1_2016', right_index = True, how = 'left').rename(columns={'SA2_NAME_2016': "DEST_SA2", 'SA3_NAME_2016': 'DEST_SA3', 'SA4_NAME_2016':'DEST_SA4', 'LGA_NAME_2016':'DEST_LGA'})


print('Number of Trip Records in Original: {}'.format(len(TRIPS.axes[0])))
print('Number of Trip Records: {}'.format(len(zone_filt.axes[0])))

trip = zone_filt
del zone_filt

Number of Trip Records in Original: 101642
Number of Trip Records: 101642


In [ ]:
%%time
"""Time Period Adjustment."""
time_filt = trip.copy()

time_filt['MidTime'] = (((time_filt['STARTIME'] + (time_filt['TRAVTIME']/2)).round(1) + 4*60)/60) %24
time_filt['MidTimeHr'] = time_filt['MidTime'].apply(np.floor)
time_filt['StartHr'] = (((time_filt['STARTIME'] + 4*60)/60) %24).apply(np.floor)
time_filt['EndHr'] = (((time_filt['STARTIME'] + time_filt['TRAVTIME'] + 4*60)/60) %24).apply(np.floor)

time_filt['MidTimeHr'].plot.hist(bins=48,alpha =0.5)
time_filt['StartHr'].plot.hist(bins=48,alpha =0.5)
time_filt['EndHr'].plot.hist(bins=48,alpha =0.5)
trip = time_filt
del time_filt


In [ ]:
%%time
"""Mode Adjustments."""
mode_filt = trip.copy()
mode_filt['MODELMODE'] = 0

#Mode Flags


mode_filt.replace({'Truck driver': 'Car driver','Truck passenger': 'Car passenger', 'Motorcycle driver': 'Car driver', 'Motorcycle passenger': 'Car passenger','Mobility scooter': 'Bicycle', 'Taxi': 'Car passenger', 'Taxi or Uber': 'Car passenger','Taxi or ride share e.g. Uber' : 'Car passenger', 'Uber / Other Ride Share' : 'Car passenger', 'Other method': 'Car driver'}, inplace = True)

car_driver = mode_filt.MAINMODE == 'Car driver'
car_pass = mode_filt.MAINMODE == 'Car passenger'
single_mode = mode_filt.MODE2 == 0

walk = mode_filt.MAINMODE == 'Walking'
bicycle = mode_filt.MAINMODE == 'Bicycle'
motorcycle_driver = mode_filt.MAINMODE == 'Motorcycle driver'
motorcycle_pass = mode_filt.MAINMODE == 'Motorcycle passenger'
truck_driver = mode_filt.MAINMODE == 'Truck driver'
truck_pass = mode_filt.MAINMODE == 'Truck passenger'
mob_scoot = mode_filt.MAINMODE == 'Mobility scooter'
taxi = mode_filt.MAINMODE == 'Taxi'
uber = mode_filt.MAINMODE == 'Taxi or Uber'
ride_share = mode_filt.MAINMODE == 'Taxi or ride share e.g. Uber'

#single modes
mode_filt.loc[car_driver, 'MODELMODE'] = 'Car driver'
mode_filt.loc[car_pass, 'MODELMODE'] = 'Car passenger'
mode_filt.loc[walk, 'MODELMODE'] = 'Walking'
mode_filt.loc[bicycle, 'MODELMODE'] = 'Bicycle'
mode_filt.loc[motorcycle_driver, 'MODELMODE'] = 'Car driver'
mode_filt.loc[motorcycle_pass, 'MODELMODE'] = 'Car passenger'
mode_filt.loc[truck_driver, 'MODELMODE'] = 'Car driver'
mode_filt.loc[truck_pass, 'MODELMODE'] = 'Car passenger'
mode_filt.loc[mob_scoot, 'MODELMODE'] = 'Bicycle'
mode_filt.loc[taxi | uber | ride_share, 'MODELMODE'] = 'Car passenger'

mode_filt.loc[truck_driver & single_mode, 'MODELMODE'] = 'Car driver'
mode_filt.loc[truck_pass & single_mode, 'MODELMODE'] = 'Car passenger'

mode_filt.replace({'Truck driver': 'Car driver','Truck passenger': 'Car passenger', 'Motorcycle driver': 'Car driver', 'Motorcycle passenger': 'Car passenger','Mobility scooter': 'Bicycle', 'Taxi': 'Car passenger', 'Taxi or Uber': 'Car passenger','Taxi or ride share e.g. Uber' : 'Car passenger', 'Uber / Other Ride Share' : 'Car passenger', 'Other method': 'Car driver'}, inplace = True)

############
# PT MODES #
############
mode_filt.replace({'Public Bus': 'Public bus','Charter/Courtesy/Other bus': 'Public bus', 'School bus (private/chartered)': 'School bus', 'School bus (with route number)': 'School bus'}, inplace = True)

#Num Stops in trip
mode_one = mode_filt.trip_stops == 0
mode_two = mode_filt.trip_stops == 1
mode_three = mode_filt.trip_stops == 2
mode_four = mode_filt.trip_stops == 3
mode_five = mode_filt.trip_stops == 4
mode_six = mode_filt.trip_stops == 5
mode_seven = mode_filt.trip_stops == 6
mode_eight = mode_filt.trip_stops == 7

#direction flag 
a_to_p = mode_filt.Direction == 'A2P'
p_to_a = mode_filt.Direction == 'P2A'
O_to_D = mode_filt.Direction == 'OD'

#PT
mode_filt.replace({'Public Bus': 'Public bus','Charter/Courtesy/Other bus': 'Public bus', 'School bus (private/chartered)': 'School bus', 'School bus (with route number)': 'School bus'}, inplace = True)
bus_public = mode_filt.MAINMODE == 'Public bus'
bus_school = mode_filt.MAINMODE == 'School bus'
train = mode_filt.MAINMODE == 'Train' 
light_rail = mode_filt.MAINMODE == 'Light rail' 
ferry = mode_filt.MAINMODE == 'Ferry' 
mode_filt['PT_access'] = 'None'
not_car = mode_filt.MAINMODE != 'Car driver'
not_car_p = mode_filt.MAINMODE != 'Car passenger'
##PNR KNR W2PT one 
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_one & empty & not_car & not_car_p, 'MODELMODE'] = mode_filt.loc[mode_one & empty, 'MAINMODE']

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_one & empty & empty_pt_ac & not_car & not_car_p, 'PT_access'] = 'Walk2PT'
##PNR KNR W2PT two 
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_two & empty & not_car & not_car_p, 'MODELMODE'] = mode_filt.loc[mode_two & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_two & empty & walk2 & not_car & not_car_p, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_two & empty & drive2 & not_car & not_car_p, 'PT_access'] = 'PnR'
mode_filt.loc[mode_two & empty & dropped2 & not_car & not_car_p, 'PT_access'] = 'KnR'

walk3 = mode_filt.MODE2 == 'Walking'
drive3 = mode_filt.MODE2 == 'Car driver'
dropped3 = mode_filt.MODE2 == 'Car passenger'
mode_filt.loc[mode_two & empty & walk3 & not_car & not_car_p, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_two & empty & drive3 & not_car & not_car_p, 'PT_access'] = 'PnR'
mode_filt.loc[mode_two & empty & dropped3 & not_car & not_car_p, 'PT_access'] = 'KnR'
empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_two & empty & empty_pt_ac & not_car & not_car_p, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT three 
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_three & empty, 'MODELMODE'] = mode_filt.loc[mode_three & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_three & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_three & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_three & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_three & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_three & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_three & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE3 == 'Walking'
drive3 = mode_filt.MODE3 == 'Car driver'
dropped3 = mode_filt.MODE3 == 'Car passenger'
mode_filt.loc[mode_three & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_three & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_three & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_three & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT four
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_four & empty, 'MODELMODE'] = mode_filt.loc[mode_four & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_four & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_four & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_four & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_four & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_four & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_four & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE4 == 'Walking'
drive3 = mode_filt.MODE4 == 'Car driver'
dropped3 = mode_filt.MODE4 == 'Car passenger'
mode_filt.loc[mode_four & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_four & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_four & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_four & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

##PNR KNR W2PT five
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_five & empty, 'MODELMODE'] = mode_filt.loc[mode_five & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_five & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_five & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_five & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_five & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_five & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_five & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE5 == 'Walking'
drive3 = mode_filt.MODE5 == 'Car driver'
dropped3 = mode_filt.MODE5 == 'Car passenger'
mode_filt.loc[mode_five & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_five & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_five & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_five & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT six
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_six & empty, 'MODELMODE'] = mode_filt.loc[mode_six & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_six & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_six & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_six & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_six & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_six & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_six & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE6 == 'Walking'
drive3 = mode_filt.MODE6 == 'Car driver'
dropped3 = mode_filt.MODE6 == 'Car passenger'
mode_filt.loc[mode_six & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_six & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_six & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_six & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT seven
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_seven & empty, 'MODELMODE'] = mode_filt.loc[mode_seven & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_seven & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_seven & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_seven & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_seven & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_seven & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_seven & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE7 == 'Walking'
drive3 = mode_filt.MODE7 == 'Car driver'
dropped3 = mode_filt.MODE7 == 'Car passenger'
mode_filt.loc[mode_seven & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_seven & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_seven & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_seven & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

In [ ]:
##PNR KNR W2PT eight
empty = mode_filt.MODELMODE == 0
mode_filt.loc[mode_eight & empty, 'MODELMODE'] = mode_filt.loc[mode_eight & empty, 'MAINMODE']

walk2 = mode_filt.MODE1 == 'Walking'
drive2 = mode_filt.MODE1 == 'Car driver'
dropped2 = mode_filt.MODE1 == 'Car passenger'
mode_filt.loc[mode_eight & empty & p_to_a & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_eight & empty & p_to_a & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_eight & empty & p_to_a & dropped2, 'PT_access'] = 'KnR'
mode_filt.loc[mode_eight & empty & O_to_D & walk2, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_eight & empty & O_to_D & drive2, 'PT_access'] = 'PnR'
mode_filt.loc[mode_eight & empty & O_to_D & dropped2, 'PT_access'] = 'KnR'
walk3 = mode_filt.MODE8 == 'Walking'
drive3 = mode_filt.MODE8 == 'Car driver'
dropped3 = mode_filt.MODE8 == 'Car passenger'
mode_filt.loc[mode_eight & empty & a_to_p & walk3, 'PT_access'] = 'Walk2PT'
mode_filt.loc[mode_eight & empty & a_to_p & drive3, 'PT_access'] = 'PnR'
mode_filt.loc[mode_eight & empty & a_to_p & dropped3, 'PT_access'] = 'KnR'

empty_pt_ac = mode_filt.PT_access == 'None'
mode_filt.loc[mode_eight & empty & empty_pt_ac, 'PT_access'] = 'Walk2PT'

del empty_pt_ac, walk2, walk3, drive2, drive3, dropped2, dropped3, empty
del a_to_p, p_to_a, O_to_D, mode_one, mode_two, mode_three, mode_four
del mode_five, mode_six, mode_seven, mode_eight
del car_driver, car_pass, motorcycle_driver, motorcycle_pass
del bicycle, bus_public, bus_school
del ferry, light_rail, mob_scoot, not_car, not_car_p, ride_share, single_mode, taxi
del train, truck_driver, truck_pass, uber, walk

print('Number of Trip Records in Original: {}'.format(len(TRIPS.axes[0])))
print('Number of Trip Records: {}'.format(len(mode_filt.axes[0])))
trip = mode_filt
del mode_filt

In [ ]:
%%time
"""GIS tagging."""
trip_prod = gpd.GeoDataFrame(trip, geometry = gpd.points_from_xy(trip['PROD_LONG'], trip['PROD_LAT']))
trip_prod = trip_prod.set_crs(zones.crs)
trip_prod = gpd.sjoin(trip_prod, zones[['geometry','LvL4_ID', 'LvL3_ID', 'LvL2_ID', 'LvL1_ID', 'Dist_ID', 'BSTM']], how = 'left', op = 'within').drop(columns ='index_right').rename(columns = {'LvL4_ID': 'PROD_LvL4', 'LvL3_ID': 'PROD_LvL3', 'LvL2_ID': 'PROD_LvL2','LvL1_ID': 'PROD_LvL1', 'Dist_ID': 'PROD_Dist', 'BSTM': 'PROD_BSTM'})
#trip_prod.to_file('tripprod.shp', driver='ESRI Shapefile')

trip_attr = gpd.GeoDataFrame(trip_prod.drop(columns = ['geometry']), geometry = gpd.points_from_xy(trip['ATTR_LONG'], trip['ATTR_LAT']))
trip_attr = trip_attr.set_crs(zones.crs)
trip_attr = gpd.sjoin(trip_attr, zones[['geometry','LvL4_ID', 'LvL3_ID', 'LvL2_ID', 'LvL1_ID', 'Dist_ID','BSTM']], how = 'left', op = 'within').drop(columns ='index_right').rename(columns = {'LvL4_ID': 'ATTR_LvL4', 'LvL3_ID': 'ATTR_LvL3', 'LvL2_ID': 'ATTR_LvL2','LvL1_ID': 'ATTR_LvL1', 'Dist_ID': 'ATTR_Dist','BSTM': 'ATTR_BSTM'})
#trip_attr[[ 'PROD_LvL4', 'PROD_LvL3', 'PROD_LvL2', 'PROD_LvL1', 'PROD_Dist', 'ATTR_LvL4', 'ATTR_LvL3', 'ATTR_LvL2', 'ATTR_LvL1', 'ATTR_Dist']]
#trip_attr.to_file('tripattr.shp', driver='ESRI Shapefile')
trip_p = pd.DataFrame(trip_attr.drop(columns = ['geometry']))
trip = trip_p

trip.loc[trip['Direction'] == 'P2A', 'Orig_Dist'] = trip.loc[trip['Direction'] == 'P2A', 'PROD_Dist']
trip.loc[trip['Direction'] == 'P2A', 'Orig_Mod'] = trip.loc[trip['Direction'] == 'P2A', 'PROD_BSTM']
trip.loc[trip['Direction'] == 'P2A', 'Dest_Dist'] = trip.loc[trip['Direction'] == 'P2A', 'ATTR_Dist']
trip.loc[trip['Direction'] == 'P2A', 'Dest_Mod'] = trip.loc[trip['Direction'] == 'P2A', 'ATTR_BSTM']

trip.loc[trip['Direction'] == 'OD', 'Orig_Dist'] = trip.loc[trip['Direction'] == 'OD', 'PROD_Dist']
trip.loc[trip['Direction'] == 'OD', 'Orig_Mod'] = trip.loc[trip['Direction'] == 'OD', 'PROD_BSTM']
trip.loc[trip['Direction'] == 'OD', 'Dest_Dist'] = trip.loc[trip['Direction'] == 'OD', 'ATTR_Dist']
trip.loc[trip['Direction'] == 'OD', 'Dest_Mod'] = trip.loc[trip['Direction'] == 'OD', 'ATTR_BSTM']

trip.loc[trip['Direction'] == 'A2P', 'Orig_Dist'] = trip.loc[trip['Direction'] == 'A2P', 'ATTR_Dist']
trip.loc[trip['Direction'] == 'A2P', 'Orig_Mod'] = trip.loc[trip['Direction'] == 'A2P', 'ATTR_BSTM']
trip.loc[trip['Direction'] == 'A2P', 'Dest_Dist'] = trip.loc[trip['Direction'] == 'A2P', 'PROD_Dist']
trip.loc[trip['Direction'] == 'A2P', 'Dest_Mod'] = trip.loc[trip['Direction'] == 'A2P', 'PROD_BSTM']

a = trip.Orig_Mod == 'Y'
b = trip.Orig_Mod == 'N'
trip.loc[a | b, 'ORIG_SEQ'] = 'Y'
trip.loc[trip.ORIG_SEQ != 'Y', 'ORIG_SEQ'] = 'N'
c = trip.Dest_Mod == 'Y'
d = trip.Dest_Mod == 'N'
trip.loc[c | d, 'DEST_SEQ'] = 'Y'
trip.loc[trip.DEST_SEQ != 'Y', 'DEST_SEQ'] = 'N'




trip['AGE5'] = (trip['AGE'] // 5 ) *5
trip['AGE10'] = (trip['AGE'] // 10 ) *10


del trip_attr, trip_prod, trip_p

In [ ]:
%%time
"""Clean Tables of non SEQ Based Households"""
Household = Household[Household.Home_Dist.notnull()]
Person = Person[Person.Home_Dist.notnull()]
trip = trip[trip.Home_Dist.notnull()]
Household = Household[Household.Home_Dist > 0]
Person = Person[Person.Home_Dist > 0]
trip = trip[trip.Home_Dist > 0]

print('Number of Household Records in Original: {}'.format(len(HOUSEHOLDS.axes[0])))
print('Number of Household Records: {}'.format(len(Household.axes[0])))
print('Number of Trip Records in Original: {}'.format(len(TRIPS.axes[0])))
print('Number of Trip Records: {}'.format(len(trip.axes[0])))
print('Number of Person Records in Original: {}'.format(len(PERSONS.axes[0])))
print('Number of Person Records: {}'.format(len(Person.axes[0])))


trip.drop(columns = ['STARTSTOP', 'STARTSTOP', 'ENDSTOP', 'MAINMODE', 'MODE1','MODE2','MODE3','MODE4', 'MODE5','MODE6', 'MODE7', 'MODE8', 'MidTime', 'ARRTIME','OVERALL_PURPOSE'], inplace = True)

a = trip.ORIG_SEQ == 'Y'
b = trip.DEST_SEQ == 'Y'
na = trip.ORIG_SEQ != 'Y'
nb = trip.DEST_SEQ != 'Y'

#int_trips = trip.loc[a & b]
#i2e_trips = trip.loc[a & nb]
#e2i_trips = trip.loc[na & b]
#e2e_trips = trip.loc[na & nb]
internal_trips = trip.loc[a | b]


del a,b,na,nb

In [ ]:
sect = wgt_sect[['Dist_ID', 'WeightingSector']]
trip = pd.merge(trip, sect, left_on = 'Home_Dist', right_on='Dist_ID', how='left').rename(columns={'WeightingSector': "Home_Sect"}).drop(columns='Dist_ID')
trip = pd.merge(trip, sect, left_on = 'PROD_Dist', right_on='Dist_ID', how='left').rename(columns={'WeightingSector': "PROD_Sect"}).drop(columns='Dist_ID')
trip = pd.merge(trip, sect, left_on = 'ATTR_Dist', right_on='Dist_ID', how='left').rename(columns={'WeightingSector': "ATTR_Sect"}).drop(columns='Dist_ID')
Person = pd.merge(Person, sect, left_on = 'Home_Dist', right_on='Dist_ID', how='left').rename(columns={'WeightingSector': "Home_Sect"}).drop(columns='Dist_ID')


In [ ]:
Person_I2I = pd.merge(Person,pd.pivot_table(internal_trips, values = ['TRIPID','TRIPWGT19'], index = ['PERSID'], columns = ['Trip_Purpose'], aggfunc={'TRIPID': 'count','TRIPWGT19':'sum'}), on ='PERSID', how = 'left' )#.fillna(0)

Person_I2I = pd.merge(Person_I2I, internal_trips.groupby(by=['PERSID']).agg({'TRIPID':'count','TRIPWGT19':'sum'}), on='PERSID', how='left')
Person_I2I.to_csv('Person_regression.csv', index=False)
Person_I2I = pd.read_csv('Person_regression.csv')

Person_I2I.fillna(0, inplace = True)
Person_I2I.columns = Person_I2I.columns.str.replace("'",'')
Person_I2I.columns = Person_I2I.columns.str.replace(",",'')
Person_I2I.columns = Person_I2I.columns.str.replace("(",'')
Person_I2I.columns = Person_I2I.columns.str.replace(")",'')
Person_I2I.columns = Person_I2I.columns.str.replace("TRIPID",'UW_')
Person_I2I.columns = Person_I2I.columns.str.replace("TRIPWGT19",'W_')
Person_I2I.columns = Person_I2I.columns.str.replace(" ",'')
Person_I2I.rename(columns={'W_':'Weighted','UW_':'Un-weighted'}, inplace= True)

"""Retroactive Person-Type Change"""
"""Taken care of by prior steps"""

In [ ]:
##Demographic Variables
non_work_flag = Person_I2I['Dependent'] == 1
worker_flag = Person_I2I['Worker'] == 1
student_flag = Person_I2I['Student'] == 1
non_student_flag = Person_I2I['Student'] != 1
worker_b_flag = Person_I2I['Worker_Blue'] == 1
worker_w_flag = Person_I2I['Worker_White'] == 1
student_t_flag = Person_I2I['Student_Tert'] == 1
student_s_flag = Person_I2I['Student_Sec'] == 1
student_p_flag = Person_I2I['Student_Prim'] == 1


Person_I2I.loc[non_work_flag & non_student_flag, 'Non-work, non-study'] = 1
Person_I2I.loc[non_work_flag & non_student_flag, 'Demog Classification' ] = 'Non-work, non-study'
Person_I2I.loc[non_work_flag & non_student_flag, 'Demog Classification2' ] = 'Non-work, non-study'

Person_I2I.loc[worker_flag & non_student_flag, 'Work, non-study'] = 1
Person_I2I.loc[worker_flag & non_student_flag, 'Demog Classification' ] = 'Work, non-study'
Person_I2I.loc[worker_flag & worker_w_flag & non_student_flag, 'Work_white, non-study'] = 1
Person_I2I.loc[worker_flag & worker_w_flag & non_student_flag, 'Demog Classification2' ] = 'Work_white, non-study'
Person_I2I.loc[worker_flag & worker_w_flag & non_student_flag, 'Work_blue, non-study'] = 1
Person_I2I.loc[worker_flag & worker_b_flag & non_student_flag, 'Demog Classification2' ] = 'Work_blue, non-study'


Person_I2I.loc[worker_flag & student_flag, 'Work, study'] = 1
Person_I2I.loc[worker_flag & student_flag, 'Demog Classification' ] = 'Work, study'
Person_I2I.loc[worker_flag & worker_w_flag & student_flag & student_t_flag, 'Work_White, study_tert'] = 1
Person_I2I.loc[worker_flag & worker_w_flag & student_flag & student_t_flag, 'Demog Classification2' ] = 'Work_white, study_tert'
Person_I2I.loc[worker_flag & worker_w_flag & student_flag & student_t_flag, 'Work_Blue, study_tert'] = 1
Person_I2I.loc[worker_flag & worker_b_flag & student_flag & student_t_flag, 'Demog Classification2' ] = 'Work_Blue, study_tert'
Person_I2I.loc[worker_flag & worker_w_flag & student_flag & student_s_flag, 'Work_White, study_sec'] = 1
Person_I2I.loc[worker_flag & worker_w_flag & student_flag & student_s_flag, 'Demog Classification2' ] = 'Work_white, study_sec'
Person_I2I.loc[worker_flag & worker_w_flag & student_flag & student_s_flag, 'Work_Blue, study_sec'] = 1
Person_I2I.loc[worker_flag & worker_b_flag & student_flag & student_s_flag, 'Demog Classification2' ] = 'Work_Blue, study_sec'

Person_I2I.loc[non_work_flag & student_flag, 'Non-work, study'] = 1
Person_I2I.loc[non_work_flag & student_flag, 'Demog Classification' ] = 'Non-work, study'
Person_I2I.loc[non_work_flag & student_flag & student_p_flag, 'Non-work, study_prim'] = 1
Person_I2I.loc[non_work_flag & student_flag & student_p_flag, 'Demog Classification2' ] = 'Non-work, study_prim'
Person_I2I.loc[non_work_flag & student_flag & student_s_flag, 'Non-work, study_sec'] = 1
Person_I2I.loc[non_work_flag & student_flag & student_s_flag, 'Demog Classification2' ] = 'Non-work, study_sec'
Person_I2I.loc[non_work_flag & student_flag & student_t_flag, 'Non-work, study_tert'] = 1
Person_I2I.loc[non_work_flag & student_flag & student_t_flag, 'Demog Classification2' ] = 'Non-work, study_tert'

trip_non_work_flag = trip['Dependent'] == 1
trip_worker_flag = trip['Worker'] == 1
trip_student_flag = trip['Student'] == 1
trip_non_student_flag = trip['Student'] != 1
trip_worker_b_flag = trip['Worker_Blue'] == 1
trip_worker_w_flag = trip['Worker_White'] == 1
trip_student_t_flag = trip['Student_Tert'] == 1
trip_student_s_flag = trip['Student_Sec'] == 1
trip_student_p_flag = trip['Student_Prim'] == 1

In [ ]:
#trip_retired_flag = trip['WORKSTATUS'] == 'retired'

trip.loc[trip_non_work_flag & trip_non_student_flag, 'Non-work, non-study'] = 1
trip.loc[trip_non_work_flag & trip_non_student_flag, 'Demog Classification' ] = 'Non-work, non-study'
trip.loc[trip_non_work_flag & trip_non_student_flag, 'Demog Classification2' ] = 'Non-work, non-study'

trip.loc[trip_worker_flag & trip_non_student_flag, 'Work, non-study'] = 1
trip.loc[trip_worker_flag & trip_non_student_flag, 'Demog Classification' ] = 'Work, non-study'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_non_student_flag, 'Work_white, non-study'] = 1
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_non_student_flag, 'Demog Classification2' ] = 'Work_white, non-study'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_non_student_flag, 'Work_blue, non-study'] = 1
trip.loc[trip_worker_flag & trip_worker_b_flag & trip_non_student_flag, 'Demog Classification2' ] = 'Work_blue, non-study'

trip.loc[trip_worker_flag & trip_student_flag, 'Work, study'] = 1
trip.loc[trip_worker_flag & trip_student_flag, 'Demog Classification' ] = 'Work, study'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_t_flag, 'Work_White, study_tert'] = 1
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_t_flag, 'Demog Classification2' ] = 'Work_white, study_tert'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_t_flag, 'Work_Blue, study_tert'] = 1
trip.loc[trip_worker_flag & trip_worker_b_flag & trip_student_flag & trip_student_t_flag, 'Demog Classification2' ] = 'Work_Blue, study_tert'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_s_flag, 'Work_White, study_sec'] = 1
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_s_flag, 'Demog Classification2' ] = 'Work_white, study_sec'
trip.loc[trip_worker_flag & trip_worker_w_flag & trip_student_flag & trip_student_s_flag, 'Work_Blue, study_sec'] = 1
trip.loc[trip_worker_flag & trip_worker_b_flag & trip_student_flag & trip_student_s_flag, 'Demog Classification2' ] = 'Work_Blue, study_sec'

trip.loc[trip_non_work_flag & trip_student_flag, 'Non-work, study'] = 1
trip.loc[trip_non_work_flag & trip_student_flag, 'Demog Classification' ] = 'Non-work, study'
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Non-work, study_prim'] = 1
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Demog Classification2' ] = 'Non-work, study_prim'
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Non-work, study_sec'] = 1
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Demog Classification2' ] = 'Non-work, study_sec'
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Non-work, study_tert'] = 1
trip.loc[trip_non_work_flag & trip_student_flag & trip_student_p_flag, 'Demog Classification2' ] = 'Non-work, study_tert'

"""Flexible Workers."""
nonflex = Person_I2I.Collar2 == "White Non-Flexible"
flex = Person_I2I.Collar2 == "White Flexible"
non_rec = Person_I2I.Collar2 == 0
white_worker = Person_I2I.Worker_White == 1

Person_I2I['Worker_White_F'] = 0
Person_I2I['Worker_White_NF'] = 0
Person_I2I.loc[white_worker & flex, 'Worker_White_F'] = 1
Person_I2I.loc[white_worker & nonflex, 'Worker_White_NF'] = 1
Person_I2I.loc[white_worker & non_rec, 'Worker_White_NF'] = 1


Person_I2I['UW_HBWW_NF'] = 0
Person_I2I['UW_HBWW_F'] = 0
Person_I2I['W_HBWW_NF'] = 0
Person_I2I['W_HBWW_F'] = 0

Person_I2I.loc[white_worker & flex,    'UW_HBWW_F'] = Person_I2I.loc[white_worker & flex,    'UW_HBWW']
Person_I2I.loc[white_worker & nonflex, 'UW_HBWW_NF'] = Person_I2I.loc[white_worker & nonflex, 'UW_HBWW']
Person_I2I.loc[white_worker & non_rec, 'UW_HBWW_NF'] = Person_I2I.loc[white_worker & non_rec, 'UW_HBWW']
Person_I2I.loc[white_worker & flex,    'W_HBWW_F'] = Person_I2I.loc[white_worker & flex,    'W_HBWW']
Person_I2I.loc[white_worker & nonflex, 'W_HBWW_NF'] = Person_I2I.loc[white_worker & nonflex, 'W_HBWW']
Person_I2I.loc[white_worker & non_rec, 'W_HBWW_NF'] = Person_I2I.loc[white_worker & non_rec, 'W_HBWW']




Person_I2I['Resident'] = 1
non_study = Person_I2I.Student == 0
Person_I2I.loc[non_study, 'Non-Study'] = 1
Person_I2I.fillna(0, inplace = True)

Person_I2I.to_csv('Person_regression.csv', index=False)
Person_I2I.to_csv('Trips_by_Person_pivot (Regression Table).csv', index=False)
Person_I2I.rename(columns ={'HH_SA2': 'SA2','HH_SA3': 'SA3','HH_SA4': 'SA4','HH_LGA': 'LGA', 'Home_Sect':'Sect'}, inplace = True)
Person_I2I.rename(columns ={'Home_LvL4': 'LvL4','Home_LvL3': 'LvL3','Home_LvL2': 'LvL2','Home_LvL1': 'LvL1','Home_Dist': 'Dist'}, inplace = True)
Person_I2I.drop(columns = ['WORKSTATUS','ANZSCO_1-digit', 'ANZSCO_3-digit', 'INDUSTRY', 'STUDYING', 'ED_TYPE', 'MAINACT', 'DEMOGWGT19'], inplace= True)

In [60]:
#Person_I2I = pd.merge(Person_I2I, wgt_sect[['Dist_ID', 'WeightingSector']], left_on='Dist', right_on = 'Dist_ID', how='left').rename(columns={'WeightingSector':'Sect'}).drop(columns=['Dist_ID'])
Person_I2I.to_csv('Person_regression.csv', index=False)

NameError: name 'Person_I2I' is not defined

In [ ]:
pivot_ui(Person_I2I, vals=['TRIPID', 'TRIPWGT19'], aggregatorName='Average', rows = ['Dep', 'Workers'], cols=['AGE10'], rendererName ='Table Barchart', outfile_path='Trips_by_Person_pivot (Regression Table).html')
os.startfile(os.path.join(adjustpath, 'Trips_by_Person_pivot (Regression Table).html'))

In [47]:
%%time
"""Export Tables"""
pivot_ui(trip, vals=['TRIPWGT19'], aggregatorName='Sum', outfile_path='Trips_pivot.html')
pivot_ui(Household, vals=['PERSWGT19'], aggregatorName='Sum', outfile_path='Households_pivot.html')
pivot_ui(Person, vals=['PERSWGT19'], aggregatorName='Sum', outfile_path='Persons_pivot.html')
trip.to_csv('Trips_table.csv')
Household.to_csv('Households_table.csv')
Person.to_csv('Persons_table.csv')


Wall time: 9.34 s
